In [2]:
import pandas as pd
import requests
import os
import shutil

In [ ]:
csv_disease=pd.read_csv("fitzpatrick17k.csv")
csv_disease.head(3)

In [ ]:
df_allergic=csv_disease[csv_disease["label"]=="allergic contact dermatitis"]
df_allergic.reset_index(inplace=True)
df_allergic["pic_name"]=df_allergic["label"].replace(' ', '_', regex=True)+"_"+df_allergic.index.astype("str")
df_allergic.head(3)

In [ ]:
dict(zip(df_allergic.pic_name, df_allergic.url))

In [ ]:
csv_disease.groupby("nine_partition_label")["md5hash"].count()

In [ ]:
csv_disease.groupby("label")["md5hash"].count().sort_values(ascending=False).head(20)

In [ ]:
csv_disease.groupby("three_partition_label")["md5hash"].count()

In [ ]:
df=csv_disease[csv_disease["nine_partition_label"]=="inflammatory"]
df

In [ ]:
df.groupby("label")["md5hash"].count().sort_values(ascending=False).to_dict()

In [ ]:
disease_list = [
    "psoriasis",
    "allergic contact dermatitis",
    ["lupus erythematosus", "lupus subacute"],
    "folliculitis",
    "scabies",
    "photodermatoses",
    ["acne vulgaris", "acne"],
    ["eczema", "dyshidrotic eczema"],
    "seborrheic dermatitis"
]

In [ ]:
def cln_data(data, data_list):
    for i in data_list:
        if isinstance(i, list):
            df = data[data["label"].isin(i)]
            disease_name = '_'.join(i)
        else:
            df = data[data["label"] == i]
            disease_name = i

        df.reset_index(inplace=True)
        df["pic_name"] = df["md5hash"]
        data_dict = dict(zip(df["pic_name"], df["url"]))

        folder_path = f"./disease/{disease_name.replace(' ', '_', )}"
        os.makedirs(folder_path, exist_ok=True)  # Create folder if it doesn't exist

        for pic_name, url in data_dict.items():
            filename = f"{pic_name.replace(' ', '_', )}.jpg"
            file_path = os.path.join(folder_path, filename)

            try:
                headers = {'User-Agent': 'Your Custom User-Agent'}
                response = requests.get(url, headers=headers)

                response.raise_for_status()  # Check for any HTTP errors

                with open(file_path, 'wb') as file:
                    file.write(response.content)

                print(f"Image downloaded: {file_path}")

            except requests.exceptions.RequestException as e:
                print(f"Error downloading image from {url}: {str(e)}")


In [ ]:
cln_data(csv_disease, disease_list)

In [17]:
csv_cancer=pd.read_csv("../Augmentation/train_df.csv")
csv_cancer

,region_ARM,region_BACK,region_CHEST,region_EAR,region_FACE,region_FOOT,region_FOREARM,region_HAND,region_LIP,region_NECK,...,cancer_history,has_piped_water,smoke,drink,gender,pesticide,hurt,diagnostic,img_id,image_path
0,0,0,0,0,0,0,0,1,0,0,...,0,1,0,0,1,0,0,ACK,PAT_711_4067_260.png,./cancer/all_cancer_images/PAT_711_4067_260.png
1,0,0,0,0,1,0,0,0,0,0,...,1,1,0,0,0,0,0,ACK,PAT_177_775_786.png,./cancer/all_cancer_images/PAT_177_775_786.png
2,0,0,0,0,0,0,1,0,0,0,...,0,1,0,0,1,0,0,ACK,PAT_747_1410_622.png,./cancer/all_cancer_images/PAT_747_1410_622.png
3,0,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,1,0,ACK,PAT_707_1327_245.png,./cancer/all_cancer_images/PAT_707_1327_245.png
4,0,0,1,0,0,0,0,0,0,0,...,1,1,1,0,1,0,0,ACK,PAT_225_345_903.png,./cancer/all_cancer_images/PAT_225_345_903.png
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,0,0,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,SCC,PAT_369_756_130.png,./cancer/all_cancer_images/PAT_369_756_130.png
596,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,SCC,PAT_338_710_319.png,./cancer/all_cancer_images/PAT_338_710_319.png
597,0,0,0,0,0,0,0,0,1,0,...,1,1,1,0,1,0,0,SCC,PAT_679_1286_677.png,./cancer/all_cancer_images/PAT_679_1286_677.png
598,0,0,0,0,0,0,1,0,0,0,...,0,1,0,1,1,0,1,SCC,PAT_93_142_269.png,./cancer/all_cancer_images/PAT_93_142_269.png


In [18]:
csv_cancer.groupby("diagnostic")["img_id"].count()

diagnostic
ACK    150
BCC    150
MEL    150
SCC    150
Name: img_id, dtype: int64

In [7]:
csv_cancer = csv_cancer[~csv_cancer["diagnostic"].isin(['SEK', 'NEV'])]
csv_cancer

,region_ARM,region_BACK,region_CHEST,region_EAR,region_FACE,region_FOOT,region_FOREARM,region_HAND,region_LIP,region_NECK,...,cancer_history,has_piped_water,smoke,drink,gender,pesticide,hurt,diagnostic,img_id,image_path
0,0,0,0,0,1,0,0,0,0,0,...,1,1,0,0,0,0,0.0,ACK,PAT_753_1427_496.png,./cancer/all_cancer_images/PAT_753_1427_496.png
1,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0.0,ACK,PAT_419_2767_323.png,./cancer/all_cancer_images/PAT_419_2767_323.png
2,1,0,0,0,0,0,0,0,0,0,...,0,1,0,0,1,0,0.0,ACK,PAT_566_178_625.png,./cancer/all_cancer_images/PAT_566_178_625.png
3,0,0,0,0,1,0,0,0,0,0,...,1,1,0,0,0,0,0.0,ACK,PAT_809_1528_118.png,./cancer/all_cancer_images/PAT_809_1528_118.png
4,0,0,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0.0,ACK,PAT_742_1402_818.png,./cancer/all_cancer_images/PAT_742_1402_818.png
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
527,1,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,0,0.0,SCC,PAT_771_1488_922.png,./cancer/all_cancer_images/PAT_771_1488_922.png
528,0,0,0,0,0,0,0,0,0,0,...,0,1,1,1,1,1,1.0,SCC,PAT_633_1199_218.png,./cancer/all_cancer_images/PAT_633_1199_218.png
529,0,0,0,0,0,0,0,0,0,1,...,0,1,0,0,1,0,0.0,SCC,PAT_330_1440_532.png,./cancer/all_cancer_images/PAT_330_1440_532.png
530,0,0,0,0,0,0,0,1,0,0,...,1,0,1,1,0,1,1.0,SCC,PAT_150_1799_644.png,./cancer/all_cancer_images/PAT_150_1799_644.png


In [19]:
def move_images(data, source_folder, destination_parent_folder):
    unique_diagnoses = data["diagnostic"].unique()

    # Create destination folders for each unique diagnosis
    for diagnosis in unique_diagnoses:
        destination_folder = os.path.join(destination_parent_folder, diagnosis)
        os.makedirs(destination_folder, exist_ok=True)

    # Move files to the respective destination folders
    for index, row in data.iterrows():
        img_id = row['img_id']
        diagnostic = row['diagnostic']
        file_name = img_id
        source_path = os.path.join(source_folder, file_name)
        destination_folder = os.path.join(destination_parent_folder, diagnostic)
        destination_path = os.path.join(destination_folder, file_name)

        if os.path.exists(source_path):
            shutil.move(source_path, destination_path)
            print(f"Moved {file_name} to {destination_folder}/{file_name}")
        else:
            print(f"File {file_name} not found in {source_folder}")

In [20]:
move_images(csv_cancer, "../zr7vgbcyr2-1/all", "../zr7vgbcyr2-1/train")

Moved PAT_711_4067_260.png to ../zr7vgbcyr2-1/train\ACK/PAT_711_4067_260.png
Moved PAT_177_775_786.png to ../zr7vgbcyr2-1/train\ACK/PAT_177_775_786.png
Moved PAT_747_1410_622.png to ../zr7vgbcyr2-1/train\ACK/PAT_747_1410_622.png
Moved PAT_707_1327_245.png to ../zr7vgbcyr2-1/train\ACK/PAT_707_1327_245.png
Moved PAT_225_345_903.png to ../zr7vgbcyr2-1/train\ACK/PAT_225_345_903.png
Moved PAT_492_1782_752.png to ../zr7vgbcyr2-1/train\ACK/PAT_492_1782_752.png
Moved PAT_492_4723_890.png to ../zr7vgbcyr2-1/train\ACK/PAT_492_4723_890.png
Moved PAT_289_442_43.png to ../zr7vgbcyr2-1/train\ACK/PAT_289_442_43.png
Moved PAT_817_4471_179.png to ../zr7vgbcyr2-1/train\ACK/PAT_817_4471_179.png
Moved PAT_202_308_721.png to ../zr7vgbcyr2-1/train\ACK/PAT_202_308_721.png
Moved PAT_571_1097_652.png to ../zr7vgbcyr2-1/train\ACK/PAT_571_1097_652.png
Moved PAT_528_992_124.png to ../zr7vgbcyr2-1/train\ACK/PAT_528_992_124.png
Moved PAT_230_1009_92.png to ../zr7vgbcyr2-1/train\ACK/PAT_230_1009_92.png
Moved PAT_753

In [3]:
csv_cancer=pd.read_csv("./metadata.csv")
csv_cancer

,patient_id,lesion_id,smoke,drink,background_father,background_mother,age,pesticide,gender,skin_cancer_history,...,diameter_2,diagnostic,itch,grew,hurt,changed,bleed,elevation,img_id,biopsed
0,PAT_1516,1765,NaN,NaN,NaN,NaN,8,NaN,NaN,NaN,...,NaN,NEV,False,False,False,False,False,False,PAT_1516_1765_530.png,False
1,PAT_46,881,False,False,POMERANIA,POMERANIA,55,False,FEMALE,True,...,5.0,BCC,True,True,False,True,True,True,PAT_46_881_939.png,True
2,PAT_1545,1867,NaN,NaN,NaN,NaN,77,NaN,NaN,NaN,...,NaN,ACK,True,False,False,False,False,False,PAT_1545_1867_547.png,False
3,PAT_1989,4061,NaN,NaN,NaN,NaN,75,NaN,NaN,NaN,...,NaN,ACK,True,False,False,False,False,False,PAT_1989_4061_934.png,False
4,PAT_684,1302,False,True,POMERANIA,POMERANIA,79,False,MALE,True,...,5.0,BCC,True,True,False,False,True,True,PAT_684_1302_588.png,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2293,PAT_1708,3156,NaN,NaN,NaN,NaN,73,NaN,NaN,NaN,...,NaN,ACK,True,False,False,False,False,False,PAT_1708_3156_175.png,False
2294,PAT_46,880,False,False,POMERANIA,POMERANIA,55,False,FEMALE,True,...,12.0,BCC,True,True,False,True,False,False,PAT_46_880_140.png,True
2295,PAT_1343,1217,NaN,NaN,NaN,NaN,74,NaN,NaN,NaN,...,NaN,SEK,False,False,False,False,False,False,PAT_1343_1217_404.png,False
2296,PAT_326,690,False,False,POMERANIA,POMERANIA,58,True,FEMALE,True,...,4.0,BCC,True,False,False,False,False,True,PAT_326_690_823.png,True


In [ ]:
csv_cancer_= csv_cancer.dropna()
"""
columns_to_fill = ["smoke", "drink", "background_father", "background_mother", "pesticide", "gender", "skin_cancer_history", "cancer_history", "has_piped_water","has_sewage_system", "fitspatrick", "diameter_1", "diameter_2"]

for column in columns_to_fill:
    mode_value = csv_cancer[column].mode()[0]
    csv_cancer[column].fillna(mode_value, inplace=True)
"""

In [ ]:
X = csv_cancer_.drop(["diagnostic", "patient_id", "lesion_id","img_id", "biopsed"], axis=1)
y = csv_cancer_["diagnostic"]

from sklearn.model_selection import train_test_split
X_encoded = pd.get_dummies(X, drop_first=True)
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, stratify=y, random_state=42)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

feature_names = X_encoded.columns
forest = RandomForestClassifier(random_state=0)
forest.fit(X_train, y_train)

In [ ]:
import time
import numpy as np

start_time = time.time()
importances = forest.feature_importances_
std = np.std([tree.feature_importances_ for tree in forest.estimators_], axis=0)
elapsed_time = time.time() - start_time

print(f"Elapsed time to compute the importances: {elapsed_time:.3f} seconds")

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

forest_importances = pd.Series(importances, index=feature_names)
forest_importances = forest_importances.sort_values(ascending=False)

top_n = 30
forest_importances = forest_importances[:top_n]
std = std[:top_n]

fig, ax = plt.subplots()
forest_importances.plot.bar(yerr=std, ax=ax)
ax.set_title("Top {} Feature Importances using MDI".format(top_n))
ax.set_ylabel("Mean Decrease in Impurity")
fig.tight_layout()
plt.show()

In [ ]:
from sklearn.inspection import permutation_importance

start_time = time.time()
result = permutation_importance(
    forest, X_test, y_test, n_repeats=10, random_state=42, n_jobs=-1
)
elapsed_time = time.time() - start_time
print(f"Elapsed time to compute the importances: {elapsed_time:.3f} seconds")

forest_importances = pd.Series(result.importances_mean, index=feature_names)
forest_importances = forest_importances.sort_values(ascending=False)

In [ ]:
fig, ax = plt.subplots()
top_n = 20
forest_importances = forest_importances[:top_n]
forest_importances.plot.bar(yerr=result.importances_std[:top_n] , ax=ax)
ax.set_title("Feature importances using permutation on full model")
ax.set_ylabel("Mean accuracy decrease")
fig.tight_layout()
plt.show()

In [ ]:
csv_cancer=pd.read_csv("./metadata.csv")
#csv_cancer=csv_cancer[["age", "lesion_id", "elevation", "has_sewage_system", "fitspatrick", "pesticide",  "skin_cancer_history","diameter_1", "diameter_2", "itch", "grew", "hurt", "bleed", "region", "gender", "skin_cancer_history", "diagnostic"]]


In [ ]:
csv_cancer            #["age", "lesion_id", "elevation", "has_sewage_system", "has_piped_water","fitspatrick", "pesticide", "background_father", "background_mother", "skin_cancer_history","diameter_1", "diameter_2", "itch", "grew", "hurt", "bleed", "region", "gender", "skin_cancer_history", "diagnostic"]

In [ ]:
import pandas as pd
import numpy as np

def filter_rows(df, diagnostic_col):
    filtered_df = df[df[diagnostic_col] != 'SEK']
    filtered_df.replace("UNK", np.nan, inplace=True)

    selected_rows = []

    grouped = filtered_df.groupby(diagnostic_col)

    for group_name, group_df in grouped:
        group_df['nan_count'] = group_df.isna().sum(axis=1)
        
        sorted_group = group_df.sort_values(by='nan_count', ascending=True)
        
        selected_rows.append(sorted_group.head(80))

    filtered_df = pd.concat(selected_rows, ignore_index=True)
    
    filtered_df = filtered_df.reset_index(drop=True)
        
    return filtered_df

In [ ]:
filtered_df = filter_rows(csv_cancer, 'diagnostic')
filtered_df

In [ ]:
filtered_df.sort_values(by='nan_count', ascending=False).head(20)

In [ ]:
filtered_df_NEV=filtered_df[filtered_df["diagnostic"]=="NEV"]
filtered_df_NEV

In [ ]:
columns_to_remove = ["lesion_id", "patient_id", "img_id", "biopsed"]
columns_name = filtered_df.columns.tolist()
for col in columns_to_remove:
    columns_name.remove(col)


In [ ]:
filtered_df.columns

In [ ]:
for column in columns_name:
    mode_value = filtered_df_NEV[column].mode()[0]
    filtered_df.loc[filtered_df["diagnostic"] == "NEV", column] = filtered_df.loc[filtered_df["diagnostic"] == "NEV", column].fillna(mode_value)

In [ ]:
import pandas as pd
from sklearn.impute import KNNImputer
from sklearn.preprocessing import OneHotEncoder

# Create a copy of the DataFrame without the columns to remove
filtered_df_copy = filtered_df.drop(columns=columns_to_remove)

# Identify categorical columns
categorical_columns = ['background_father', 'background_mother']

# Convert categorical columns to one-hot encoded representation
encoder = OneHotEncoder()
filtered_df_encoded = pd.get_dummies(filtered_df_copy, columns=categorical_columns)

# Separate categorical and numerical columns
numerical_columns = filtered_df_encoded.select_dtypes(include='number').columns

# Initialize the KNN imputer
knn_imputer = KNNImputer(n_neighbors=5)

# Fit and transform the numerical columns with the KNN imputer
imputed_values = knn_imputer.fit_transform(filtered_df_encoded[numerical_columns])

# Convert the imputed values array back to a DataFrame
imputed_df = pd.DataFrame(imputed_values, columns=numerical_columns)

# Update the NaN values in the original DataFrame with the imputed values
filtered_df.update(imputed_df)


In [ ]:
filtered_df['nan_count'] = filtered_df.isna().sum(axis=1)
filtered_df.sort_values(by='nan_count', ascending=False).head(20)

In [ ]:
filtered_df=filtered_df[["age", "lesion_id", "elevation", "has_sewage_system", "fitspatrick", "pesticide",  "skin_cancer_history","diameter_1", "diameter_2", "itch", "grew", "hurt", "bleed", "region", "gender", "diagnostic"]]
filtered_df

In [ ]:
filtered_df_cat=["elevation", "has_sewage_system", "pesticide",  "skin_cancer_history", "itch", "grew", "hurt", "bleed", "region", "gender"]
filtered_df_num=["age", "lesion_id", "fitspatrick", "diameter_1", "diameter_2"]

features=filtered_df_cat+filtered_df_num

target_variable = 'diagnostic'

In [ ]:
X = filtered_df.drop(["diagnostic"], axis=1)
y = filtered_df["diagnostic"]

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)

In [ ]:
# data pre-processing stack
from sklearn.preprocessing import MinMaxScaler,OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
import time

In [ ]:
numerical_transformer = Pipeline(steps=[('scaler', StandardScaler())])

In [ ]:
categorical_transformer = Pipeline(steps=[('one-hot-encode', OneHotEncoder(drop='first'))])

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[("num", numerical_transformer, filtered_df_num),("cat", categorical_transformer, filtered_df_cat)])

In [ ]:
estimator = Pipeline(steps=[('preprocessor', preprocessor),('rf', RandomForestRegressor())])
param_grid = {'rf__n_estimators': [200,300,500,1000],
              'rf__max_depth': [5,10,20,50,100],
              'rf__min_samples_split': [2, 5, 11]
             }

In [ ]:
gscv = GridSearchCV(estimator=estimator, param_grid=param_grid, scoring='r2', cv=5, n_jobs=-1, verbose=1)

In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.fit_transform(y_test)

In [ ]:
X_train

In [ ]:
y.isna().sum()

In [ ]:
ti = time.time()
gscv.fit(X_train, y_train_encoded)
tf = time.time()

In [ ]:
gscv.best_params_

In [ ]:
gscv.best_score_

In [ ]:
best_model_1 = gscv.best_estimator_
best_model_1

In [ ]:
training_score_1 = best_model_1.score(X_train,y_train_encoded)

test_score_1 = best_model_1.score(X_test,y_test_encoded)

print(f'Train score: {round(training_score_1,6)}')
print(f'Test score : {round(test_score_1,6)}')